In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Input, Lambda, Dense, Flatten
from sklearn.utils import shuffle
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import glob
from mtcnn import MTCNN

In [39]:
path = "Desktop/Clear Mask/"

In [40]:
X = []
y = []
size = (96,96)

In [6]:
detector = MTCNN()

Instructions for updating:
Colocations handled automatically by placer.


In [41]:
n=113
i=0
while(i<n):
    file = "Mask"+str(i)+".png"
    file_path = path + file
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    face = detector.detect_faces(img)
    for image in face:
        x1, y1, width, height = image["box"]
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        face_image = img[y1:y2, x1:x2]
        face_image = cv2.resize(face_image, size)
        X.append(face_image)
        y.append("Mask ON")
    i+=1

In [42]:
print(np.shape(X))

(126, 96, 96, 3)


In [43]:
len(X)

126

In [44]:
len(y)

126

In [46]:
n=50
i=0
path = "Desktop/gender_dataset_face/"
while(i!=n):
    file = "man/img"+str(i)+".jpg"
    file_path = path + file
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    face = detector.detect_faces(img)
    for image in face:
        x1, y1, width, height = image["box"]
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        face_image = img[y1:y2, x1:x2]
        face_image = cv2.resize(face_image, size)
        X.append(face_image)
        y.append("Mask OFF")
    file = "woman/img"+str(i)+".jpg"
    file_path = path + file
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, size)
    face = detector.detect_faces(img)
    for image in face:
        x1, y1, width, height = image["box"]
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        face_image = img[y1:y2, x1:x2]
        face_image = cv2.resize(face_image,size)
        X.append(face_image)
        y.append("Mask OFF")
    i+=1

In [47]:
print(np.shape(X))

(223, 96, 96, 3)


In [48]:
X = shuffle(X,random_state = 7)
y = shuffle(y,random_state = 7)

In [49]:
X = np.array(X)
y = np.array(y)

In [50]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y_onehot = np_utils.to_categorical(y,2)

In [52]:
X_train,X_test,y_train,y_test = train_test_split(X,y_onehot,stratify=y)

In [53]:
vgg16 =  VGG16(input_shape=(96,96,3), weights='imagenet', include_top=False)

In [54]:
for layer in vgg16.layers:
    layer.trainable = False

In [55]:
x = Flatten()(vgg16.output)
prediction = Dense(2, activation='softmax')(x)

In [61]:
mask_model = Model(inputs=vgg16.input, outputs=prediction)
mask_model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0   

In [62]:
mask_model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [63]:
history = mask_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 167 samples, validate on 56 samples
Epoch 1/10
167/167 [==============================] - 21s 124ms/step - loss: 0.1930 - accuracy: 0.9880 - val_loss: 0.2878 - val_accuracy: 0.9821
Epoch 2/10
167/167 [==============================] - 20s 118ms/step - loss: 0.1718 - accuracy: 0.9880 - val_loss: 0.7164 - val_accuracy: 0.9107
Epoch 3/10
167/167 [==============================] - 20s 118ms/step - loss: 0.2242 - accuracy: 0.9820 - val_loss: 0.4816 - val_accuracy: 0.9643
Epoch 4/10
167/167 [==============================] - 20s 117ms/step - loss: 0.0965 - accuracy: 0.9940 - val_loss: 0.2893 - val_accuracy: 0.9821
Epoch 5/10
167/167 [==============================] - 20s 118ms/step - loss: 0.0965 - accuracy: 0.9940 - val_loss: 0.2878 - val_accuracy: 0.9821
Epoch 6/10
167/167 [==============================] - 20s 117ms/step - loss: 0.0965 - accuracy: 0.9940 - val_loss: 0.2878 - val_accuracy: 0.9821
Epoch 7/10
167/167 [==============================] - 23s 139ms/step - loss: 0.0965 -

In [64]:
mask_model.save_weights("Desktop/Mask_detection.model")

In [66]:
encoder.inverse_transform(y)[0]

'Mask OFF'

In [67]:
y[0]

0